In [1]:
import requests
import pandas as pd
from secret import API_KEY
from data import DATA

In [2]:
champ = "Vayne"

In [3]:
def getJsonFromlolApi(country, g):
    res = requests.get(f"https://{country}.api.riotgames.com/lol/match/v5/matches/{g}?api_key={API_KEY}")
    try:
        if res.status_code != 200: raise Exception
        return res.json()
    except Exception:
        print(res.json()["status"])
        SystemExit("Stop right there!")

In [4]:
def getRecentGames(puuid, country):
    return requests.get(f"https://{country}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=60&count=30&api_key={API_KEY}").json()

In [5]:
def getDFfromJson(champ):
    array = []
    for p in DATA[champ]["players"]:
        games = getRecentGames(p["puuid"], p["country"])
        
        for g in games:
            json = getJsonFromlolApi(p['country'], g)
            index = json['metadata']['participants'].index(p["puuid"])
            player = json['info']['participants'][index]
            if player['championName'] != champ: continue
            if player['teamPosition'] != DATA[champ]["role"]: continue
            player['gameDuration'] = json['info']['gameDuration']
            try:
                player['hadAfkTeammate'] = player['challenges']['hadAfkTeammate']
            except KeyError:
                player['hadAfkTeammate'] = 0
            array.append(pd.DataFrame({x: [player[x]] for x in DATA[champ]["columns"]}))

    return pd.concat(array).reset_index(drop=True)

In [6]:
df = getDFfromJson(champ)

In [7]:
df['scoreCreeps'] = df['totalMinionsKilled'] + df['neutralMinionsKilled']

In [8]:
df.to_csv(f"./{champ}3.txt", index=False, header=False)